In [203]:
import random
import string
import numpy as np
import pandas as pd
from tabulate import tabulate

### Config

In [257]:
# How many dishes are available and in what range of calories
DISH_COUNT = 300
MIN_CALORIES_PER_DISH = 100
MAX_CALORIES_PER_DISH = 1500

# User config - how many days, how many dishes per day, how many calories per day
DAYS_PER_MENU = 7
DISHES_PER_DAY = 3
CALORIES_PER_DAY = 2000
# Penalty for dishes repaeating during the week
PENALTY = 800

# Genetic algorithm config
POPULATION_SIZE = 20
MATING_POPULATION_SIZE = 10
ITERATIONS = 1000

#### Helper functions

In [258]:
def visualize_menu(menu):
    print("Score: " + str(score_menu(menu)))
    headers = ["Day", "Calories", "Dishes"]
    calories = count_calories_per_day(menu)
    table = []
    for day in range(DAYS_PER_MENU):
        table.append([day, calories[day], menu[day]])
    print(tabulate(table, headers, tablefmt="fancy_grid"))

#### Real data

In [259]:
dishes = {}
data = pd.read_csv('data.csv', delimiter=',', encoding='utf-8', low_memory=False)
df = pd.DataFrame(data, columns=['name', 'Food Group', "calories_portion"])

while len(dishes) != DISH_COUNT:
    random_choice = df.sample()
    if MIN_CALORIES_PER_DISH < random_choice.calories_portion.values[0] < MAX_CALORIES_PER_DISH:
        dishes[random_choice.name.values[0]] = random_choice.calories_portion.values[0]
dishes = list(dishes.items())
print(*dishes[:7], sep='\n')

('Palm Hearts Cooked Assume Fat Not Added In Cooking', 166.44)
('Red Kidney Beans Canned Drained Made With Oil', 338.4)
('Cheese Sandwich American Cheese On White Bread No Spread', 288.66)
('Rice Fried With Pork', 354.42)
('Beef Chuck Under Blade Pot Roast Or Steak Boneless Separable Lean Only Trimmed To 0 Inch Fat Choice Raw', 123.25)
('Pie Pecan Commercially Prepared', 115.588)
('Parsnips Creamed', 239.4)


#### Sample random menu

In [260]:
def create_random_menu():
    menu = np.ndarray((DAYS_PER_MENU, DISHES_PER_DAY), tuple)
    for day in range(DAYS_PER_MENU):
        for dish_index in range(DISHES_PER_DAY):
            menu[day, dish_index] = random.choice(dishes)
    return menu

In [261]:
def count_calories_per_day(menu):

    calories = []
    for day in range(DAYS_PER_MENU):
        calories.append(0)
        for dish_index in range(DISHES_PER_DAY):
            calories[-1] += menu[day, dish_index][1]
    
    return calories

def count_duplicates_per_day(menu):
    
    names = np.reshape(menu, DAYS_PER_MENU * DISHES_PER_DAY)
    unique, counts = np.unique(names, return_counts=True)
    return 0 if max(counts) == 1 else max(counts)

def score_menu(menu):
    
    differences = [calories - CALORIES_PER_DAY for calories in count_calories_per_day(menu)]
    squares = [calories ** 2 for calories in differences]
    
    return sum(squares) + (count_duplicates_per_day(menu) * PENALTY) ** 2

In [262]:
sample_menu = create_random_menu()
visualize_menu(sample_menu)

Score: 14544040.553599998
╒═══════╤════════════╤════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╕
│   Day │   Calories │ Dishes                                                                                                                             │
╞═══════╪════════════╪════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╡
│     0 │     599.36 │ [('Mixed Vegetables Cooked From Canned Made With Butter', 127.16)                                                                  │
│       │            │  ('Caribou Eye Raw (Alaska Native)', 326.0)                                                                                        │
│       │            │  ('Turkey Fryer-Roasters Meat And Skin Cooked Roasted', 146.2)]                                                                    │
├───────┼────────────┼────────────────

In [263]:
def crossover(first_menu, second_menu):
    child = first_menu.copy()
    for day_index in range(DAYS_PER_MENU):
        if random.choice((0, 1)) == 1:
            child[day_index] = second_menu[day_index]
    return child


def random_mutate(menu):
    menu[random.randint(0, DAYS_PER_MENU - 1), random.randint(0, DISHES_PER_DAY - 1)] = random.choice(dishes)
    return menu


def swap_mutate(menu):
    first_day_index = random.randint(0, DAYS_PER_MENU - 1)
    second_day_index = random.randint(0, DAYS_PER_MENU - 1)
    dish_index = random.randint(0, DISHES_PER_DAY - 1)
    menu[first_day_index, dish_index], menu[second_day_index, dish_index] = menu[second_day_index, dish_index], menu[
        first_day_index, dish_index]
    return menu

In [264]:
population = []
for _ in range(POPULATION_SIZE):
    population.append(create_random_menu())

In [265]:
for i in range(ITERATIONS):
    population.sort(key=score_menu)
    mating_population = population[:MATING_POPULATION_SIZE]

    offspring_population = []
    for _ in range(POPULATION_SIZE - MATING_POPULATION_SIZE):
        offspring_population.append(crossover(random.choice(mating_population), random.choice(mating_population)))
        if random.random() < 0.1:
            offspring_population[-1] = random_mutate(offspring_population[-1])
        if random.random() < 0.1:
            offspring_population[-1] = swap_mutate(offspring_population[-1])

    population = mating_population + offspring_population

In [266]:
def check_duplicant(menu):
    names = np.reshape(menu, 21)
    unique, counts = np.unique(names, return_counts=True)
    is_repetable = dict(zip(unique, counts))
    
    for elem in is_repetable:
        if is_repetable[elem] > 1:
            print("duplicants detected")
            return
    print("No duplicants")

    
population.sort(key=score_menu)
visualize_menu(population[0])    
check_duplicant(population[0])

Score: 62174.83739999996
╒═══════╤════════════╤══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╕
│   Day │   Calories │ Dishes                                                                                                                                                   │
╞═══════╪════════════╪══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╡
│     0 │    1841.19 │ [('Sugar Brown And Water Syrup', 747.78)                                                                                                                 │
│       │            │  ('Seeds Breadnut Tree Seeds Dried', 587.2) ('Beef Pot Pie', 506.21)]                                                                                    │
├───────┼────────────┼───────────────────────────────────────────────────────────────